In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

import gridstatus

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [4]:
ISOS = ["ERCOT"]

In [5]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

13it [00:01,  7.69it/s]


In [6]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

ERCOT


In [7]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [8]:
for iso in ISOS:
    clean_final(iso, "all_data")

                               hour  regdn  regup    rrs  nspin  \
time_tomerge                                                      
2014-01-01 06:00:00+00:00      1:00   5.65   9.72  11.72   1.00   
2014-01-01 07:00:00+00:00      2:00   5.55   8.22   8.73   1.00   
2014-01-01 08:00:00+00:00      3:00   5.55   7.57   8.11   1.00   
2014-01-01 09:00:00+00:00      4:00   8.11   4.81   7.25   1.00   
2014-01-01 10:00:00+00:00      5:00   8.11  10.00  10.00   1.00   
...                             ...    ...    ...    ...    ...   
2019-01-01 01:00:00+00:00     20:00   4.01   4.00   9.24   0.89   
2019-01-01 02:00:00+00:00     21:00   3.50   4.00   7.21   1.28   
2019-01-01 03:00:00+00:00     22:00   4.01   3.22   6.02   0.94   
2019-01-01 04:00:00+00:00     23:00   7.15   6.18   6.38   0.97   
2019-01-01 05:00:00+00:00  24:00:00   4.58   4.00   4.61   0.97   

                           quantity_nsrs  quantity_regdn  quantity_regup  \
time_tomerge                                        